In [1]:
%%writefile crud_path.py
# crud_path.py
from fastapi import FastAPI, HTTPException

# Create a FastAPI instance
app = FastAPI()

# User database
USER_DB = {}

# Fail response
NAME_NOT_FOUND = HTTPException(status_code=400, detail="Name not found.")


@app.post("/users/name/{name}/nickname/{nickname}")
def create_user(name: str, nickname: str):
    USER_DB[name] = nickname
    return {"status": "success"}


@app.get("/users/name/{name}")
def read_user(name: str):
    if name not in USER_DB:
        raise NAME_NOT_FOUND
    return {"nickname": USER_DB[name]}


@app.put("/users/name/{name}/nickname/{nickname}")
def update_user(name: str, nickname: str):
    if name not in USER_DB:
        raise NAME_NOT_FOUND
    USER_DB[name] = nickname
    return {"status": "success"}


@app.delete("/users/name/{name}")
def delete_user(name: str):
    if name not in USER_DB:
        raise NAME_NOT_FOUND
    del USER_DB[name]
    return {"status": "success"}


Writing crud_path.py


In [2]:
%%writefile crud_query.py
# crud_query.py
from fastapi import FastAPI, HTTPException

# Create a FastAPI instance
app = FastAPI()

# User database
USER_DB = {}

# Fail response
NAME_NOT_FOUND = HTTPException(status_code=400, detail="Name not found.")


@app.post("/users")
def create_user(name: str, nickname: str):
    USER_DB[name] = nickname
    return {"status": "success"}


@app.get("/users")
def read_user(name: str):
    if name not in USER_DB:
        raise NAME_NOT_FOUND
    return {"nickname": USER_DB[name]}


@app.put("/users")
def update_user(name: str, nickname: str):
    if name not in USER_DB:
        raise NAME_NOT_FOUND
    USER_DB[name] = nickname
    return {"status": "success"}


@app.delete("/users")
def delete_user(name: str):
    if name not in USER_DB:
        raise NAME_NOT_FOUND
    del USER_DB[name]
    return {"status": "success"}


Writing crud_query.py


## API 테스트

---

`http://localhost:8000/docs`를 통해 FastAPI 의 Swagger UI 에 접속하여 다음의 시나리오가 잘 작동하는지 확인합니다.

### Create
    - POST 를 통해 `{"name": "hello", "nickname": "world"}` 로 create 합니다.
        - `{"status": "success"}` 를 정상적으로 return 하는지 확인합니다.

### Read
    - GET 을 통해 `{"name": "hello"}` 로 read 합니다.
        - `{"nickname": "world"}` 를 정상적으로 return 하는지 확인합니다.
    - GET 을 통해 `{"name": "hello2"}` 로 read 합니다.
        - `{"detail": "Name not found."}` 를 return 하며 400 error 가 발생하는지 확인합니다.

### Update
    - PUT 을 통해 `{"name": "hello", "nickname": "world"}` 로 update 합니다.
        - `{"status": "success"}` 를 정상적으로 return 하는지 확인합니다.
        - GET 을 통해 `{"name": "hello"}` 로 read 하여 정상적으로 `{"nickname": "world"}` 를 return 하는지 확인합니다.
    - PUT 을 통해 `{”name”: “hello”, “nickname”: “world2”}` 로 update 합니다.
        - `{"status": "success"}` 를 정상적으로 return 하는지 확인합니다.
        - GET 을 통해 `{"name": "hello"}` 로 read 하여 정상적으로 `{"nickname": "world2"}` 를 return 하는지 확인합니다.
    - PUT 을 통해 `{"name": "hello2", "nickname": "world2"}` 로 update 합니다.
        - `{"detail": "Name not found."}` 를 return 하며 400 error 가 발생하는지 확인합니다.

Delete
    - DELETE 를 통해 `{"name": "hello"}` 로 delete 합니다.
        - `{"status": "success"}` 를 정상적으로 return 하는지 확인합니다.
        - GET 을 통해 `{"name": "hello"}` 로 read 하여 `{"detail": "Name not found."}` 를 return 하며 400 error 가 발생하는지 확인합니다.
    - DELETE 를 통해 `{"name": "hello2"}` 로 delete 합니다.
        - `{"detail": "Name not found."}` 를 return 하며 400 error 가 발생하는지 확인합니다.

## Path Parameter vs Query Parameter

---

- Path Parameter 를 이용한 API 에서는 path 에 변수의 값을 저장하여 함수에 전달합니다.

- 반면, Query Parameter 를 이용한 API 에서는 path 에 변수의 값을 저장하지 않고 데이터를 전달합니다.

# FastAPI CRUD (Pydantic)

In [3]:
%%writefile crud_pydantic.py
# crud_pydantic.py
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel


class CreateIn(BaseModel):
    name: str
    nickname: str


class CreateOut(BaseModel):
    status: str
    id: int

# Create a FastAPI instance
app = FastAPI()

# User database
USER_DB = {}

# Fail response
NAME_NOT_FOUND = HTTPException(status_code=400, detail="Name not found.")


@app.post("/users", response_model=CreateOut)
def create_user(user: CreateIn):
    USER_DB[user.name] = user.nickname
    user_dict = user.dict()
    user_dict["status"] = "success"
    user_dict["id"] = len(USER_DB)
    return user_dict


@app.get("/users")
def read_user(name: str):
    if name not in USER_DB:
        raise NAME_NOT_FOUND
    return {"nickname": USER_DB[name]}


@app.put("/users")
def update_user(name: str, nickname: str):
    if name not in USER_DB:
        raise NAME_NOT_FOUND
    USER_DB[name] = nickname
    return {"status": "success"}


@app.delete("/users")
def delete_user(name: str):
    if name not in USER_DB:
        raise NAME_NOT_FOUND
    del USER_DB[name]
    return {"status": "success"}

Writing crud_pydantic.py
